In [6]:
import json
import requests
import csv 

api_key_drew = 'sBfMQZWan1rpr3ugchVmpKLV_mIw9-GUMnNF52cznlGZc7GINf2J3TBP0We9j8fyIPFMsZP_if7E6H0k72AvPsMU8wwL-hOXWbW2zuRkSEa4TmBEb171AGvtbOecXXYx'
api_key_erik = 'L9JXqxrohAsyw83Y87m4Aw1IpMjOOkhw660ZaZXxpYi5cwgJD2p_EG7nbfzmE827BmPNkpZ-R3ch6J91e1thjTILnGyAV5fWygEDCx1imp-eqVMlQBHZzGSUfx-uXXYx'


In [7]:
def parse_yelp_business_data(businesses, params):
    csvData = []
    for business in businesses: 
        data = []
        for param in params: 
            try:
                if (param == 'coordinates'):
                    coordinates = business[param]
                    data.append(coordinates['latitude'])
                    data.append(coordinates['longitude'])
                elif (param == 'location'):
                    location = business[param]
                    address = location['address1']
                    city = location['city']
                    zipcode = location['zip_code']
                    state = location['state']
                    country = location['country']
                    data.extend([address, city, zipcode, state, country])
                elif (param == 'categories'):
                    categories = business[param]
                    cats = []
                    for category in categories:
                        cats.append(category['alias'])
                    data.append(','.join([str(x) for x in cats]))
                else:    
                    value = business[param]
                    data.append(value)
            except:
                data.append('MISSING')
        if ( len(data) != 0):
            csvData.append(data)
    return csvData

In [8]:
def get_parsed_json_yelp_data(api_key, term, borough, limit, offset):
    headers = {'Authorization': 'Bearer %s' % api_key}
    url = 'https://api.yelp.com/v3/businesses/search'
    params = {'term': term, 'location': borough, 'limit': limit, 'offset': offset}
    req = requests.get(url, params=params, headers=headers)
    parsed = json.loads(req.text)
    return parsed

In [9]:
def create_empty_csv_file(name, header): 
    with open(name, 'w', newline='', encoding='utf-8') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerows([header])
    csvFile.close()
    
def append_to_csv_file(name, csvData):
    with open(name, 'a', newline='', encoding='utf-8') as csvFile:
        writer = csv.writer(csvFile)
        try: 
            writer.writerows(csvData)
        except Exception as e:
            print(csvData)
            print(e)
    csvFile.close()

In [10]:
params = ['id','alias','name','is_closed','review_count','rating','price',
#          'transactions',
          'categories',         
          'coordinates',
          'location']

header = ['id','alias','name','is_closed','review_count','rating','price',
#          'transactions',
          'categories',
          'latitude','longitude',
          'address','city','zipcode','state','country']

name = 'restaurant_data.csv'
term = 'Restaurants'
borough = 'Brooklyn, NY'
offset = 900
max_limit = 50 ##TODO change to 50
limit = max_limit

create_empty_csv_file(name, header)

parsed = get_parsed_json_yelp_data(api_key_erik, term, borough, limit, offset)
total = parsed['total']
print("Total Number of Records: {}".format(total))

businesses = parsed['businesses']
csvData = parse_yelp_business_data(businesses,params)
print("Offset: {}, Total: {}, Businesses: {}".format(offset, total, len(businesses)))
append_to_csv_file(name, csvData)

offset += limit
while (offset < total): 
    try: 
        parsed = get_parsed_json_yelp_data(api_key_erik, term, borough, limit, offset)
        businesses = parsed['businesses']
        csvData = parse_yelp_business_data(businesses, params)
        print("Offset: {}, Total: {}, Businesses: {}".format(offset, total, len(businesses)))
        append_to_csv_file(name,csvData)
        offset += limit
    except:
        print("Issue while appending parsed data: {}".format(parsed))
        break


KeyError: 'total'